In [135]:
import os
import pandas as pd
from extraction import FemTransform
import settings
import numpy as np
import func

In [138]:
func.list_files_in_folder(settings.fem_folder_save)


Files in "/Users/ivanov.ev/Work/Dashboard/fem/results" displayed below
--------------
1. extraction_09102022.xlsx
--------------



['extraction_09102022.xlsx']

In [142]:
data = pd.ExcelFile(os.path.join(settings.fem_folder_save,
                                 'extraction_09102022.xlsx')).parse(settings.fem_sheet_name)

In [143]:
data.head()

,programme,code,project,typecf,subtypecf,dzo,year,value,input_file
0,УПРАВЛЕНИЕ ВЗАИМООТНОШЕНИЯМИ С ПОДРЯДЧИКАМИ БРД,U200001737,«Развитие Системы рейтингования»,Затраты,Капитальные затраты (CAPEX):,ПАО Газпром нефть,2020,2.099999,УВП.xlsm
1,УПРАВЛЕНИЕ ВЗАИМООТНОШЕНИЯМИ С ПОДРЯДЧИКАМИ БРД,U200001737,«Развитие Системы рейтингования»,Затраты,Капитальные затраты (CAPEX):,ПАО Газпром нефть,2021,0.921878,УВП.xlsm
2,УПРАВЛЕНИЕ ВЗАИМООТНОШЕНИЯМИ С ПОДРЯДЧИКАМИ БРД,U220002107,Личный кабинет подрядчика 2.0,Затраты,Операционные затраты (OPEX):капитализируемые,ПАО Газпром нефть,2022,5.600000,УВП.xlsm
3,УПРАВЛЕНИЕ ВЗАИМООТНОШЕНИЯМИ С ПОДРЯДЧИКАМИ БРД,U220002107,Личный кабинет подрядчика 2.0,Затраты,Операционные затраты (OPEX):,ПАО Газпром нефть,2022,3.800000,УВП.xlsm
4,УПРАВЛЕНИЕ ВЗАИМООТНОШЕНИЯМИ С ПОДРЯДЧИКАМИ БРД,NaN,Уберизация закупок БРД,Затраты,Операционные затраты (OPEX):,ПАО Газпром нефть,2022,3.800000,УВП.xlsm


In [166]:
today_year = 2022
far_future_after = 2025

bins = [2022, 2025, 2030]
bins_labels = ['бл. будущее', 'далекое будущее']

data_slice = data[['year', 'value']]

In [167]:
data_slice['new'] = pd.cut(data_slice['year'], bins, labels=bins_labels)

/var/folders/c4/8t91mr493fv14ypfd7mxbcc00000gn/T/ipykernel_76206/4107957821.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_slice['new'] = pd.cut(data_slice['year'], bins, labels=bins_labels)


In [162]:
data_slice

,year,value,new
0,2020,2.099999,NaN
1,2021,0.921878,NaN
2,2022,5.600000,NaN
3,2022,3.800000,NaN
4,2022,3.800000,NaN
...,...,...,...
8717,2029,14.708174,далекое будущее
8718,2029,10.237489,далекое будущее
8719,2029,1.320750,далекое будущее
8720,2029,117.435409,далекое будущее


In [8]:
import pandas as pd
import os
import settings
import func


def report_process():
    pass


def choose_excel_file(folder_path, sheet_name=0):
    file_path = os.path.join(folder_path, func.single_input_file(folder_path))
    result = pd.ExcelFile(file_path).parse(sheet_name)
    return result


class Reports:

    def __init__(self, fem_folder_results, mapping_folder_results):

        self.fem: pd.DataFrame = choose_excel_file(fem_folder_results, sheet_name=settings.fem_sheet_name)
        self.mapping: pd.DataFrame = choose_excel_file(mapping_folder_results, sheet_name=settings.mapping_sheet_name)
        self.result = self.fem.copy(deep=True)

    def apply_mapping_to_fem(self, fillna=True):
        mapping_column_names = ['query', 'chosen']
        columns_in_mapping = self.mapping.column.unique().tolist()

        for fem_column in self.fem.columns:
            if fem_column in columns_in_mapping:
                temp_mapping = self.mapping.loc[self.mapping['column'] == fem_column,
                                                mapping_column_names].copy()

                temp_dict = (temp_mapping
                             .set_index(mapping_column_names[0])[mapping_column_names[1]]
                             .to_dict()
                             )
                self.result[fem_column] = self.result[fem_column].map(temp_dict)
                if fillna:
                    self.result[fem_column] = (self.result[fem_column]
                                               .map(temp_dict)
                                               .fillna(self.result[fem_column])
                                               )
                else:
                    self.result[fem_column] = self.result[fem_column].map(temp_dict)

            get_column_fem = self.fem[self.fem['']]
            get_column_mapping = self.mapping[self.mapping['column'] == fem_column]

            return self







In [14]:
a = Reports(settings.fem_folder_results, settings.mapping_folder_results)


Files in "/Users/ivanov.ev/Work/Dashboard/fem/results" displayed below
--------------
1. extraction_10102022.xlsx
2. extraction_09102022.xlsx
--------------
Choose one file (input a number)
Blank values are not allowed.
Blank values are not allowed.
Blank values are not allowed.

Files in "/Users/ivanov.ev/Work/Dashboard/mapping/results" displayed below
--------------
1. mapping_11102022.xlsx
2. mapping_10102022.xlsx
3. mapping_09102022.xlsx
--------------
Choose one file (input a number)


In [13]:
a.mapping

,query,result,matched,score,column,chosen
0,УПРАВЛЕНИЕ ВЗАИМООТНОШЕНИЯМИ С ПОДРЯДЧИКАМИ БРД,Управление взаимоотношениями с подрядчиками,matched,84.59,programme,УВП
1,Долгосрочное развитие БРД,Долгосрочное развитие,matched,82.94,programme,АБ1: ДР
2,Долгосрочное развитие БРД,Долгосрочное развитие,matched,82.94,programme,АБ1: ДР
3,Pro-Развитие,PRO-Развитие,matched,99.00,programme,Pro-Р
4,Управление данными БРД,Управление данными,matched,82.50,programme,УПД
...,...,...,...,...,...,...
109,Оптимизация затрат,Оптимизация затрат,matched,99.00,subtypecf,Сокращение opex
110,Повышение доходов от оказания услуг либо прода...,Повышение доходов от оказания услуг либо прода...,matched,99.00,subtypecf,Сторонние доходы
111,Дополнительный доход,Дополнительный доход,matched,99.00,subtypecf,Изменение ДП от УВ
112,Налог,Налог,matched,99.00,subtypecf,tax


In [12]:
a.fem

,programme,code,project,typecf,subtypecf,dzo,year,value,input_file
0,ТОРО,U190001328,ИТ компонент ТОРО Управление жизненным циклом ...,Затраты,Капитальные затраты (CAPEX):,КЦ,2020,131.168000,ТОРО.xlsx
1,ТОРО,U190001328,ИТ компонент ТОРО Управление жизненным циклом ...,Затраты,Операционные затраты (OPEX):,КЦ,2020,6.895000,ТОРО.xlsx
2,ТОРО,U190001328,ИТ компонент ТОРО Управление жизненным циклом ...,Затраты,ННП,КЦ,2020,1.537000,ТОРО.xlsx
3,ТОРО,U200001878,Тиражирование Фазы 1 ТОРО БРД на ГПН-Хантос,Затраты,Операционные затраты (OPEX):,КЦ,2020,0.395000,ТОРО.xlsx
4,ТОРО,U200001879,Тиражирование Фазы 1 ТОРО БРД на ГПН-Оренбург,Затраты,Операционные затраты (OPEX):,КЦ,2020,0.395000,ТОРО.xlsx
...,...,...,...,...,...,...,...,...,...
8717,Формирование бизнес-кейсов БРД,NaN,ОИЗ (Хоппер),ТРИЭ,NaN,АО Мессояханефтегаз,2030,226.270466,АБ2.xlsm
8718,Формирование бизнес-кейсов БРД,NaN,ОИЗ (Хоппер),ТРИЭ,NaN,ООО Газпромнефть-Восток,2030,15.471797,АБ2.xlsm
8719,Формирование бизнес-кейсов БРД,NaN,ОИЗ (Хоппер),ТРИЭ,NaN,ООО Газпромнефть-Ямал,2030,26.773409,АБ2.xlsm
8720,Формирование бизнес-кейсов БРД,NaN,Новые опции,ТРИЭ,NaN,"ОАО Славнефть-Мегионнефтегаз, ОАО Славнефть-Ме...",2030,0.496540,АБ2.xlsm


In [15]:
a.fem.columns

Index(['programme', 'code', 'project', 'typecf', 'subtypecf', 'dzo', 'year',
       'value', 'input_file'],
      dtype='object')

In [17]:
for i in a.fem.columns:
    print(i)

programme
code
project
typecf
subtypecf
dzo
year
value
input_file


In [19]:
test = a.mapping.loc[:, ['query', 'chosen']]

In [32]:
map = pd.Series(data=a.mapping['query'], index=a.mapping['chosen'])

In [99]:
fem = a.fem.copy()
mapping = a.mapping.copy()

mapping_column_names = ['query', 'chosen']
temp_mapping = mapping.loc[mapping['column'] == 'programme',
                           mapping_column_names].copy()


In [100]:
test_dict = (temp_mapping[mapping_column_names]
             .set_index(mapping_column_names[0])[mapping_column_names[1]]
             .to_dict()
             )
for i in ['ИМА', 'ТОРО', 'ИРиОС', 'Экосистема ', 'УПРАВЛЕНИЕ ВЗАИМООТНОШЕНИЯМИ  С ПОДРЯДЧИКАМИ БРД']:
    test_dict.pop(i)

test_dict

{'Долгосрочное развитие БРД': 'АБ1: ДР',
 'Долгосрочное развитие БРД ': 'АБ1: ДР',
 'Pro-Развитие': 'Pro-Р',
 'Управление данными БРД': 'УПД',
 'Управление добычей на текущих мощностях': 'АБ5: УДТМ',
 'Когнитивная геология': 'КГ',
 'Программа цифровизации газового бизнеса': 'ГБ',
 'Цифровизация Сбыта Газа': 'CГ',
 'АБ4: Реализация цикла ввода новых мощностей добычи': 'АБ4: РЦВНМ',
 'Цифровая энергетика': 'ЦЭ',
 'Цифровая экономика БРД': 'ЦЭк',
 'Проектирование ввода мощностей': 'АБ3: ПВM',
 'Формирование бизнес-кейсов БРД': 'АБ2: ФБК',
 'ДТР': 'ДТР',
 'ИПА': 'ИПА'}

In [101]:
fem['programme'] = fem['programme'].map(test_dict).fillna(fem['programme'])
fem

,programme,code,project,typecf,subtypecf,dzo,year,value,input_file
0,ТОРО,U190001328,ИТ компонент ТОРО Управление жизненным циклом ...,Затраты,Капитальные затраты (CAPEX):,КЦ,2020,131.168000,ТОРО.xlsx
1,ТОРО,U190001328,ИТ компонент ТОРО Управление жизненным циклом ...,Затраты,Операционные затраты (OPEX):,КЦ,2020,6.895000,ТОРО.xlsx
2,ТОРО,U190001328,ИТ компонент ТОРО Управление жизненным циклом ...,Затраты,ННП,КЦ,2020,1.537000,ТОРО.xlsx
3,ТОРО,U200001878,Тиражирование Фазы 1 ТОРО БРД на ГПН-Хантос,Затраты,Операционные затраты (OPEX):,КЦ,2020,0.395000,ТОРО.xlsx
4,ТОРО,U200001879,Тиражирование Фазы 1 ТОРО БРД на ГПН-Оренбург,Затраты,Операционные затраты (OPEX):,КЦ,2020,0.395000,ТОРО.xlsx
...,...,...,...,...,...,...,...,...,...
8717,АБ2: ФБК,NaN,ОИЗ (Хоппер),ТРИЭ,NaN,АО Мессояханефтегаз,2030,226.270466,АБ2.xlsm
8718,АБ2: ФБК,NaN,ОИЗ (Хоппер),ТРИЭ,NaN,ООО Газпромнефть-Восток,2030,15.471797,АБ2.xlsm
8719,АБ2: ФБК,NaN,ОИЗ (Хоппер),ТРИЭ,NaN,ООО Газпромнефть-Ямал,2030,26.773409,АБ2.xlsm
8720,АБ2: ФБК,NaN,Новые опции,ТРИЭ,NaN,"ОАО Славнефть-Мегионнефтегаз, ОАО Славнефть-Ме...",2030,0.496540,АБ2.xlsm


In [102]:
fem.programme.unique().tolist()

['ТОРО',
 'ИРиОС',
 'УПРАВЛЕНИЕ ВЗАИМООТНОШЕНИЯМИ  С ПОДРЯДЧИКАМИ БРД',
 'АБ1: ДР',
 'ИМА',
 'Pro-Р',
 'ДТР',
 'УПД',
 'АБ5: УДТМ',
 'Экосистема ',
 'ИПА',
 'КГ',
 'ГБ',
 'CГ',
 'АБ4: РЦВНМ',
 'ЦЭ',
 'ЦЭк',
 'АБ3: ПВM',
 'АБ2: ФБК']